In [ ]:
# This cell tells python to use the local version of CVToolkit instead of the installed one.
# Use for testing purposes until branch feature/BCV-970-oor-metrics is merged.
import sys
import os

module_path = os.path.abspath(os.path.join("../../CVToolkit"))
if module_path not in sys.path:
    sys.path.insert(0, module_path)

In [ ]:
import pathlib
import sys

sys.path.append("../..")

from objectherkenning_openbare_ruimte.performance_evaluation_pipeline.metrics import tba_calculator
from objectherkenning_openbare_ruimte.data_minimalisation_pipeline.source import blurring_tools

In [ ]:
import cv2

data_folder = "../../datasets/oor/data-minimalisation"
output_folder = "../../datasets/oor/data-minimalisation/blurred"

blur_kernel_size_outside = 55
blur_kernel_size_inside = 165
blur_outside_padding = 25
crop_padding = 25

blur_inside = True
blur_outside = False
crop = False
draw_box = True

In [ ]:
img_file = pathlib.Path("../../datasets/oor/data-minimalisation/images/TMX7316010203-001145_pano_0000_000618_right.png")

filename = img_file.stem
label_file = os.path.join(data_folder, "labels", filename + ".txt")

image = cv2.imread(img_file.as_posix(), cv2.IMREAD_COLOR)

if not os.path.isfile(label_file):
    print("Nothing to blur!")
else:
    with open(label_file, "r") as file:
        lines = sorted(file.readlines())
        for line in lines:
            yolo_annotation = line.strip()
            class_id = int(yolo_annotation.split(sep=" ", maxsplit=1)[0])
            if blur_inside and (class_id in (0, 1)):
                image = blurring_tools.blur_inside_yolo_box(
                    image, yolo_annotation, blur_kernel_size_inside)
            if blur_outside and (class_id in (2,)):
                image = blurring_tools.blur_outside_yolo_box(
                    image, yolo_annotation, blur_kernel_size_outside, blur_outside_padding)
            if crop and (class_id in (2,)):
                image = blurring_tools.crop_outside_yolo_box(image, yolo_annotation, crop_padding)
            if draw_box and (class_id in (2,)):
                image = blurring_tools.draw_yolo_box(image, yolo_annotation)

In [ ]:
cv2.imshow('Blurred image', image)
cv2.waitKey(0)
cv2.destroyAllWindows()

In [ ]:
cv2.imwrite("../../datasets/oor/data-minimalisation/scenario_A.jpg", image)

## Collect sample data

In [ ]:
# Find all images with containers

import os
import pandas as pd

from typing import List

def get_labels(label_file: str) -> List[int]:
    labels: List[int] = [0]*5
    if not os.path.isfile(label_file):
        return labels
    with open(label_file, "r") as file:
        lines = file.readlines()
        for line in lines:
            label = int(line.split(sep=" ")[0])
            labels[label] += 1
    return labels

data_folder = "../../datasets/oor/first-train-oor-nc5"

data = {
    "filename": [],
    "n_person": [],
    "n_license_plate": [],
    "n_container": [],
}

images = list(pathlib.Path(os.path.join(data_folder, "images", "val")).glob("*.png"))

for img in images:
    filename = img.stem
    label_file = os.path.join(data_folder, "labels", "val", filename + ".txt")
    labels = get_labels(label_file)
    data["filename"].append(filename)
    data["n_person"].append(labels[0])
    data["n_license_plate"].append(labels[1])
    data["n_container"].append(labels[2])

df = pd.DataFrame(data=data)

In [ ]:
import shutil

filenames = df.query("n_container > 0 & (n_license_plate > 0 | n_person > 0)")["filename"].to_list()

target_dir = "../../datasets/oor/data-minimalisation"

pathlib.Path(os.path.join(target_dir, "images")).mkdir(parents=True, exist_ok=True)
pathlib.Path(os.path.join(target_dir, "labels")).mkdir(parents=True, exist_ok=True)

for file in filenames:
    img_src = os.path.join(data_folder, "images", "val", file + ".png")
    img_dst = os.path.join(target_dir, "images", file + ".png")
    shutil.copyfile(img_src, img_dst)
    lab_src = os.path.join(data_folder, "labels", "val", file + ".txt")
    lab_dst = os.path.join(target_dir, "labels", file + ".txt")
    shutil.copyfile(lab_src, lab_dst)